# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,15.57,71,100,2.68,NZ,1738809341
1,1,adamstown,-25.0660,-130.1015,25.27,78,17,5.15,PN,1738809342
2,2,pilar,-9.5972,-35.9567,23.96,91,100,0.75,BR,1738809343
3,3,isafjordur,66.0755,-23.1240,0.88,83,100,19.84,IS,1738809211
4,4,vorgashor,67.5833,63.9500,-14.42,98,100,3.83,RU,1738809346


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    tiles="OSM",
    scale=1,
    color="City",
    alpha=0.7,
    frame_width=700,
    frame_height=500,
    title="City Humidity Levels"
)

# Display the map
map_plot

c:\Users\aaron\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\aaron\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\aaron\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"]>= 15.6) & (city_data_df["Max Temp"] <= 26.6) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] < 20) &
    (city_data_df["Humidity"] < 65)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,rosso,16.4986,-15.8153,18.25,17,0,5.21,SN,1738809361
18,18,margaret river,-33.9500,115.0667,26.14,48,0,7.35,AU,1738809363
31,31,port lincoln,-34.7333,135.8667,24.28,55,1,4.92,AU,1738809379
81,81,bardai,11.6500,7.2333,16.89,13,0,4.16,NG,1738809438
90,90,el porvenir,13.7617,-87.3458,23.01,59,6,2.89,HN,1738809450


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,rosso,SN,16.4986,-15.8153,17,
18,margaret river,AU,-33.9500,115.0667,48,
31,port lincoln,AU,-34.7333,135.8667,55,
81,bardai,NG,11.6500,7.2333,13,
90,el porvenir,HN,13.7617,-87.3458,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rosso - nearest hotel: نزل صانتر روصو
margaret river - nearest hotel: Margaret River Hotel
port lincoln - nearest hotel: Boston Hotel
bardai - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
damboa - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
ratnagiri - nearest hotel: Vihar
codrington - nearest hotel: No hotel found
mae chan - nearest hotel: ต้นห้ารีสอร์ท
whangarei - nearest hotel: Grand Hotel
raichur - nearest hotel: Hotel Nrupatunga
al qadarif - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
tura - nearest hotel: Hotel Polo Orchid
uthal - nearest hotel: No hotel found
sidhi - nearest hotel: No hotel found
plaza huincul - nearest hotel: Antu Malal Aparts & Suites
frontera - nearest hotel: Quirino
muang xay - nearest hotel: ໂຮງແຮມງາມລາວ
kiffa - nearest hotel: فندق كيفة
dwarka - nearest hotel: The Dwar

,City,Country,Lat,Lng,Humidity,Hotel Name
16,rosso,SN,16.4986,-15.8153,17,نزل صانتر روصو
18,margaret river,AU,-33.9500,115.0667,48,Margaret River Hotel
31,port lincoln,AU,-34.7333,135.8667,55,Boston Hotel
81,bardai,NG,11.6500,7.2333,13,No hotel found
90,el porvenir,HN,13.7617,-87.3458,59,No hotel found
92,tshabong,BW,-26.0500,22.4500,44,No hotel found
121,vallenar,CL,-28.5708,-70.7581,56,Hotel del Marqués
140,damboa,NG,11.1560,12.7576,16,No hotel found
236,crane,US,31.3974,-102.3501,57,No hotel found
248,ratnagiri,IN,16.9833,73.3000,38,Vihar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    frame_width=700,
    frame_height=500,
    title="Hotels in Ideal Vacation Cities",
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)